# 填充和步幅
在上⼀节的例⼦⾥，我们使⽤⾼和宽为3的输⼊与⾼和宽为2的卷积核得到⾼和宽为2的输出。⼀
般来说，假设输⼊形状是$n_h × n_w$，卷积核窗口形状是$k_h × k_w$，那么输出形状将会是

$(n_h − k_h + 1) × (n_w − k_w + 1)$

所以卷积层的输出形状由输⼊形状和卷积核窗口形状决定。本节我们将介绍卷积层的两个超参
数，即填充和步幅。它们可以对给定形状的输⼊和卷积核改变输出形状。
## 填充
填充（padding）是指在输⼊⾼和宽的两侧填充元素（通常是0元素）。图5.2⾥我们在原输⼊⾼和宽的
两侧分别添加了值为0的元素，使得输⼊⾼和宽从3变成了5，并导致输出⾼和宽由2增加到4。图5.2中
的阴影部分为第⼀个输出元素及其计算所使⽤的输⼊和核数组元素： 0×0+0×1+0×2+0×3 = 0。

![在输入的高和宽两侧分别填充了0元素的二维互相关计算](../img/conv_pad.svg)

⼀般来说，如果在⾼的两侧⼀共填充$p_h$⾏，在宽的两侧⼀共填充$p_w$列，那么输出形状将会是

$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1),$

也就是说，输出的⾼和宽会分别增加$p_h和p_w$。

在很多情况下，我们会设置$p_h = k_h−1和p_w = k_w−1$来使输⼊和输出具有相同的⾼和宽。这样会⽅
便在构造⽹络时推测每个层的输出形状。假设这⾥$k_h$是奇数，我们会在⾼的两侧分别填充$p_h/2$⾏。
如果$k_h$是偶数，⼀种可能是在输⼊的顶端⼀侧填充$\lceil ph/2 \rceil$⾏，而在底端⼀侧填充$\lfloor ph/2\rfloor$⾏。在宽
的两侧填充同理。

卷积神经⽹络经常使⽤奇数⾼宽的卷积核，如1、 3、 5和7，所以两端上的填充个数相等。对任意
的⼆维数组X，设它的第i⾏第j列的元素为X[i,j]。当两端上的填充个数相等，并使输⼊和输出
具有相同的⾼和宽时，我们就知道输出Y[i,j]是由输⼊以X[i,j]为中⼼的窗口同卷积核进⾏
互相关计算得到的。

下⾯的例⼦⾥我们创建⼀个⾼和宽为3的⼆维卷积层，然后设输⼊⾼和宽两侧的填充数分别为1。
给定⼀个⾼和宽为8的输⼊，我们发现输出的⾼和宽也是8。

In [1]:
from mxnet import nd
from mxnet.gluon import nn

# 定义一个函数来计算卷积层。它初始化卷积层权重，并对输入和输出做相应的升维和降维
def comp_conv2d(conv2d, X):
	conv2d.initialize()
	# (1, 1)代表批量大小和通道数（“多输入通道和多输出通道”一节将介绍）均为1
	X = X.reshape((1, 1) + X.shape)
	Y = conv2d(X)
	return Y.reshape(Y.shape[2:])  # 排除不关心的前两维：批量和通道

# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d = nn.Conv2D(1, kernel_size=3, padding=1)
X = nd.random.uniform(shape=(8, 8))
comp_conv2d(conv2d, X).shape

(8, 8)

当卷积核的⾼和宽不同时，我们也可以通过设置⾼和宽上不同的填充数使输出和输⼊具有相同的
⾼和宽。

In [2]:
# 使⽤⾼为5、宽为3的卷积核。在⾼和宽两侧的填充数分别为2和1
conv2d = nn.Conv2D(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

(8, 8)